# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
!python create_tables.py
!python etl.py

73 files found in data/song_data
1/73 files processed.
2/73 files processed.
3/73 files processed.
4/73 files processed.
5/73 files processed.
6/73 files processed.
7/73 files processed.
8/73 files processed.
9/73 files processed.
10/73 files processed.
11/73 files processed.
12/73 files processed.
13/73 files processed.
14/73 files processed.
15/73 files processed.
16/73 files processed.
17/73 files processed.
18/73 files processed.
19/73 files processed.
20/73 files processed.
21/73 files processed.
22/73 files processed.
23/73 files processed.
24/73 files processed.
25/73 files processed.
26/73 files processed.
27/73 files processed.
28/73 files processed.
29/73 files processed.
30/73 files processed.
31/73 files processed.
32/73 files processed.
33/73 files processed.
34/73 files processed.
35/73 files processed.
36/73 files processed.
37/73 files processed.
38/73 files processed.
39/73 files processed.
40/73 files processed.
41/73 files processed.
42/73 files processed.
43/73 file

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [ ]:
df =  pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
test = df[['song_id','title','artist_id','year','duration']]
print(list(zip(*map(test.get, test))))


In [ ]:
song_data = list(zip(*map(test.get, test)))
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
try:
    cur.execute(song_table_insert, song_data[0])
    conn.commit()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the DB")
    print(e)

In [ ]:
import io
def export(df, filepath,title):
    buffer = io.StringIO()
    df.info(buf=buffer);
    s = buffer.getvalue()
    with open(filepath, "w",encoding="utf-8") as f:
        f.write(title+'\n')
        f.write(s[37:])
export(test,"results/songstable.txt", title='Song Table')

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
df.head()

In [ ]:
test = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
export(test,"results/artisttable.txt", title='Artists Table')
test = list(zip(*map(test.get, test)))

In [ ]:
artist_data = test[0]
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the DB")
    print(e)



Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files('data/log_data')

In [ ]:
filepath = log_files[0]

In [ ]:
df = pd.read_json(filepath,lines=True)
df.ts = pd.to_datetime(df["ts"])
df.head()

In [ ]:
export(df,"results/logtable.txt", title='Log Table')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df.page == 'NextSong']
df.head()


In [ ]:
t = pd.to_datetime(df["ts"], unit='ms')
df["ts"] = pd.to_datetime(df["ts"], unit='ms')
t.head()

In [ ]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ("time","hour","day","week","month","year","weekday")


In [ ]:
i = 0
for t,h,d,w,m,y,we in zip(df.ts, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday):
    print(t,h,d,w,m,y,we)
    i += 1
    break

In [ ]:
temp = dict()
for col_lbl, values in zip(column_labels,time_data):
    temp[col_lbl] = values.values.flatten().tolist()
time_df = pd.DataFrame(temp)
time_df.time = pd.to_datetime(time_df.time)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
time_df.dtypes

In [ ]:
for i, row in time_df.iterrows():
#     print(tuple(row))
    cur.execute(time_table_insert, tuple(row))
    conn.commit()

In [ ]:
export(time_df,"results/timetable.txt", title='Time Table')

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [ ]:
export(user_df,"results/usertable.txt", title='User Table')

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
songplay_dict = {"ts":[], "userId":[], "level":[], "songid":[], "artistid":[], "sessionId":[], "location":[], "userAgent":[]}
def update_dic(row, songid, artistid):
    songplay_dict["ts"].append(row.ts)
    songplay_dict["userId"].append(row.userId)
    songplay_dict["level"].append(row.level)
    songplay_dict["songid"].append(songid)
    songplay_dict["artistid"].append(artistid)
    songplay_dict["sessionId"].append(row.sessionId)
    songplay_dict["location"].append(row.location)
    songplay_dict["userAgent"].append(row.userAgent)
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
#     (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    update_dic(row, songid, artistid)
    songplay_data =  (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
df_songplay = pd.DataFrame(songplay_dict)

In [ ]:
export(df_songplay,"results/songsplaytable.txt", title='Songsplay Table')

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.